In [ ]:
import subprocess
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import icepack

In [ ]:
surface_filename = "greenland-surface.tif"
if not pathlib.Path(surface_filename).is_file():
    bedmachine_filename = icepack.datasets.fetch_bedmachine_greenland()
    command = f"rio convert netcdf:{bedmachine_filename}:surface {surface_filename}"
    subprocess.run(command.split())

bounds = {
    "left": -256.6e3, "bottom": -2345.8e3, "right": -79.3e3, "top": -2215.8e3,
}

with rasterio.open(surface_filename, "r") as source:
    transform = source.transform
    window = (
        rasterio.windows.from_bounds(**bounds, transform=transform)
        .round_lengths()
        .round_offsets()
    )
    surface = source.read(indexes=1, window=window)
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    transform = rasterio.windows.transform(window, transform)

extent = (xmin, xmax, ymin, ymax)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("easting (meters)")
ax.set_ylabel("northing (meters)")
ax.set_title("Jakobshavn Isbrae surface elevation")
colors = ax.imshow(surface, cmap="Blues_r", extent=extent)
fig.colorbar(colors, label="meters");

In [ ]:
velocity_filename = "greenland-v.tif"
if not pathlib.Path(velocity_filename).is_file():
    measures_filenames = icepack.datasets.fetch_measures_greenland()
    filenames = [[f for f in measures_filenames if val in f][0] for val in ("vx", "vy")]
    for filename, val in zip(filenames, ("vx", "vy")):
        command = f"rio warp {filename} greenland-{val}.tif --like {surface_filename}"
        subprocess.run(command.split())

    command = f"rio stack greenland-vx.tif greenland-vy.tif {velocity_filename}"
    subprocess.run(command.split())

with rasterio.open(velocity_filename, "r") as source:
    velocity = source.read(window=window, masked=True)

In [ ]:
from matplotlib import colors as clrs
speed = np.sqrt(velocity[0]**2 + velocity[1]**2)

fig, ax = plt.subplots()
ax.set_xlabel("easting (meters)")
ax.set_ylabel("northing (meters)")
ax.set_title("Jakobshavn Isbrae speed")
colors = ax.imshow(speed, cmap="Blues", extent=extent, norm=clrs.LogNorm(vmin=1.0, vmax=14e3))
fig.colorbar(colors, label="meters/year");

In [ ]:
ds_dx = np.zeros_like(surface)
ds_dy = np.zeros_like(surface)
ds_dx[:, 1:-1] = (surface[:, 2:] - surface[:, :-2]) / transform.a
ds_dy[1:-1, :] = (surface[2:, :] - surface[:-2, :]) / transform.e

In [ ]:
slope = np.sqrt(ds_dx**2 + ds_dy**2)

fig, ax = plt.subplots()
ax.set_xlabel("easting (meters)")
ax.set_ylabel("northing (meters)")
ax.set_title("Jakobshavn Isbrae surface slope")
colors = ax.imshow(slope, vmax=0.25, cmap="Blues", extent=extent)
fig.colorbar(colors);

In [ ]:
product = velocity[0] * ds_dx + velocity[1] * ds_dy

vmin, vmax = -1e3, +1e3
fig, ax = plt.subplots()
ax.set_xlabel("easting (meters)")
ax.set_ylabel("northing (meters)")
ax.set_xlim((-200e3, -150e3))
ax.set_ylim((-2300e3, -2240e3))
ax.set_title("$u\\cdot\\nabla s$")
colors = ax.imshow(product, extent=extent, vmin=vmin, vmax=vmax, cmap="RdBu")
fig.colorbar(colors);

In [ ]:
xs = np.linspace(xmin, xmax, surface.shape[1])
ys = np.linspace(ymax, ymin, surface.shape[0])

fig, ax = plt.subplots()
ax.set_xlabel("easting (meters)")
ax.set_ylabel("northing (meters)")
ax.set_xlim((-200e3, -150e3))
ax.set_ylim((-2300e3, -2240e3))
colors = ax.imshow(speed, cmap="Blues", extent=extent, norm=clrs.LogNorm(vmin=1.0, vmax=14e3))
ax.contour(xs, ys, surface, colors="k", linewidths=0.5, levels=np.linspace(0.0, 2000.0, 21));